### Check an endpoint
This notebook aims to run the pipeline on a given endpoint to check to see if it will be successful. This includes collecting, pipeline and datset stages. It aims to also highlight useful information as a summary as to whether the endpoint would be successful on our platform. it will download all relevant data to do this and hence might be disk intensive. You'll need to provide the following information:
- collection - this is the collection that the dataset belongs to, this can be extracted from the specification but for this notebook we ask to provide it incase you want to test the pipeline on something which isn't being included in the main site right now
- dataset - this is the dataset that the endpoint is meant to provide data for, technically this can be multiple datasets but this this use case it should just be one. It is also the name of the pipeline that is ran on the individual resources that are downloaded from the endpoint. the terms dataset/pipline are often the same
- organisation - the organisation identifier to be used for the endpoint
- endpoint url - the actual url needed for the endpoint
- plugin - often we use plugins to download the data this is only needed for specific endpoints

If you are seeing errors regarding digital-land, then try 

`pip install -e git+https://github.com/digital-land/pipeline.git#egg=digital-land`

And restart the notebook.

In [17]:
import os
import pandas as pd
from functions import run_endpoint_workflow
from sqlite_query_functions import DatasetSqlite
from convert_functions import convert_resource
from digital_land.collection import Collection

In [18]:
#southwark convservation area endpoint
# collection_name = 'conservation-area-collection'
# dataset = 'conservation-area'
# organisation = 'local-authority-eng:SWK'
# endpoint_url = 'https://www.southwark.gov.uk/assets/attach/194104/Conservation-Areas.gpkg'
# plugin = None
# additional_column_mappings=None
# additional_concats=None

# doncaster tpos
# collection_name = 'tree-preservation-order-collection'
# dataset = 'tree-preservation-zone'
# organisation = 'local-authority-eng:DNC'
# endpoint_url='https://maps.doncaster.gov.uk/server/rest/services/Planning/TPO_Map/MapServer/1'
# plugin = 'arcgis'
# additional_column_mappings=None
# additional_concats=None

# 
collection_name = 'listed-building-collection'
dataset = 'listed-building-outline'
organisation = 'local-authority-eng:CAM'
endpoint_url = 'https://opendata.camden.gov.uk/api/views/uu3n-zgbj/rows.csv?accessType=DOWNLOAD'
plugin = None

additional_column_mappings=None
additional_concats=None

# generic data_dir setting
data_dir = '../data/endpoint_checker'

# example playing with additional confiigs
# additional_concats = [{
#     'dataset':'tree-preservation-zone',
#     'endpoint':'de1eb90a8b037292ef8ae14bfabd1184847ef99b7c6296bb6e75379e6c1f9572',
#     'resource':'e6b0ccaf9b50a7f57a543484fd291dbe43f52a6231b681c3a7cc5e35a6aba254',
#     'field':'reference',
#     'fields':'REFVAL;LABEL',
#     'separator':'/'
# }]


In [19]:
run_endpoint_workflow(
    collection_name,
    dataset,
    organisation,
    endpoint_url,
    plugin,
    data_dir,
    additional_col_mappings=additional_column_mappings,
    additional_concats=additional_concats
)

HTTP Error 404: Not Found
HTTP Error 404: Not Found
../data/endpoint_checker/collection
[{'resource': 'bf55d2bbd7f121f5bddff6caa683d09422232b04159ed01db4e8f3922e2d2660', 'bytes': '', 'endpoints': '431efe2f2cbc73eb181f67ce9cc27cc61f88236835eb2b3b96ee6a54ceb6458a', 'organisations': 'local-authority-eng:CAM', 'datasets': 'listed-building-outline', 'start-date': '2023-09-26', 'end-date': ''}]


In [20]:
# validate endpoint


#### collection log summaries

We need to establish if a resource was downloaded from the endpoint and whether there were any issues during the collection process. Examine the output of the below. There should be one log for the attempt made at downloading from the endpoint. If status code is 200 then the resource was downloaded successfully

In [21]:
collection = Collection(os.path.join(data_dir,'collection'))
collection.load(directory=os.path.join(data_dir,'collection'))
logs = collection.log.entries
logs = pd.DataFrame.from_records(logs)
logs

,bytes,content-type,elapsed,endpoint,resource,status,entry-date,start-date,end-date,exception
0,,text/csv; charset=utf-8,0.482,431efe2f2cbc73eb181f67ce9cc27cc61f88236835eb2b...,bf55d2bbd7f121f5bddff6caa683d09422232b04159ed0...,200,2023-09-26T16:29:38.039471,,,


### Check unnassigned entiities
This process automatically aims to detect and assign entity numbers where entries are currently unnassigned. Examine the list below to see what (if any) entities have been assigned. if you were to include these in an actual pipeline you would need to update the configuration lookup.csv with these values. It's worth checking they are sensible before this happens 

In [22]:
unassigned_entries = pd.read_csv(os.path.join(data_dir,'var','cache','unassigned-entries.csv'))
if len(unassigned_entries) == 0:
    print('no additional entity numbers where required')
else:
    print(unassigned_entries)

                 organisation                   prefix reference
0     local-authority-eng:CAM  listed-building-outline    LB1859
1     local-authority-eng:CAM  listed-building-outline    LB1481
2     local-authority-eng:CAM  listed-building-outline     LB173
3     local-authority-eng:CAM  listed-building-outline    LB1872
4     local-authority-eng:CAM  listed-building-outline    LB1531
...                       ...                      ...       ...
1956  local-authority-eng:CAM  listed-building-outline    LB1983
1957  local-authority-eng:CAM  listed-building-outline    LB1985
1958  local-authority-eng:CAM  listed-building-outline    LB1986
1959  local-authority-eng:CAM  listed-building-outline    LB1987
1960  local-authority-eng:CAM  listed-building-outline    LB1448

[1961 rows x 3 columns]


#### Check logs collated from the pipeline process
We need to readin in logs and examine to see if the data points were all read in correctly This uses the sqlite database to do so with some custom queries I have wrote. you could directly examine the csvs if the pipeline fails.

first check column mappings to see what columns the pipeline automatically mapped. if this is empty or missing values it's likely to be the reason data isn't appearing at the end.

In [23]:
dataset_db = DatasetSqlite(os.path.join(data_dir,'dataset',f'{dataset}.sqlite3'))
results = dataset_db.get_column_mappings()
results

,end_date,entry_date,field,dataset,start_date,resource,column
0,,2023-09-26T16:31:41Z,end-date,listed-building-outline,,bf55d2bbd7f121f5bddff6caa683d09422232b04159ed0...,end-date
1,,2023-09-26T16:31:41Z,entry-date,listed-building-outline,,bf55d2bbd7f121f5bddff6caa683d09422232b04159ed0...,entry-date
2,,2023-09-26T16:31:41Z,geometry,listed-building-outline,,bf55d2bbd7f121f5bddff6caa683d09422232b04159ed0...,geometry
3,,2023-09-26T16:31:41Z,listed-building,listed-building-outline,,bf55d2bbd7f121f5bddff6caa683d09422232b04159ed0...,listed-building
4,,2023-09-26T16:31:41Z,listed-building-grade,listed-building-outline,,bf55d2bbd7f121f5bddff6caa683d09422232b04159ed0...,listed-building-grade
5,,2023-09-26T16:31:41Z,name,listed-building-outline,,bf55d2bbd7f121f5bddff6caa683d09422232b04159ed0...,name
6,,2023-09-26T16:31:41Z,notes,listed-building-outline,,bf55d2bbd7f121f5bddff6caa683d09422232b04159ed0...,notes
7,,2023-09-26T16:31:41Z,reference,listed-building-outline,,bf55d2bbd7f121f5bddff6caa683d09422232b04159ed0...,reference
8,,2023-09-26T16:31:41Z,start-date,listed-building-outline,,bf55d2bbd7f121f5bddff6caa683d09422232b04159ed0...,start-date


examine the issues logs, we'll look at the types of errors being raised and list all of them. This could be improved in the future by examining the severity

In [24]:
dataset_db = DatasetSqlite(os.path.join(data_dir,'dataset',f'{dataset}.sqlite3'))
results = dataset_db.get_issues_by_type()
results

,issue_type,count
0,Unexpected geom type,64
1,Unexpected geom type within GeometryCollection,79
2,default-value,3922
3,invalid geometry,5


In [25]:
results = dataset_db.get_issues()
results

,end_date,entry_date,entry_number,field,issue_type,line_number,dataset,resource,start_date,value
0,,,1,organisation,default-value,2,listed-building-outline,bf55d2bbd7f121f5bddff6caa683d09422232b04159ed0...,,local-authority-eng:CAM
1,,,1,entry-date,default-value,2,listed-building-outline,bf55d2bbd7f121f5bddff6caa683d09422232b04159ed0...,,2023-09-26
2,,,2,organisation,default-value,3,listed-building-outline,bf55d2bbd7f121f5bddff6caa683d09422232b04159ed0...,,local-authority-eng:CAM
3,,,2,entry-date,default-value,3,listed-building-outline,bf55d2bbd7f121f5bddff6caa683d09422232b04159ed0...,,2023-09-26
4,,,3,organisation,default-value,4,listed-building-outline,bf55d2bbd7f121f5bddff6caa683d09422232b04159ed0...,,local-authority-eng:CAM
...,...,...,...,...,...,...,...,...,...,...
4065,,,1959,entry-date,default-value,1960,listed-building-outline,bf55d2bbd7f121f5bddff6caa683d09422232b04159ed0...,,2023-09-26
4066,,,1960,organisation,default-value,1961,listed-building-outline,bf55d2bbd7f121f5bddff6caa683d09422232b04159ed0...,,local-authority-eng:CAM
4067,,,1960,entry-date,default-value,1961,listed-building-outline,bf55d2bbd7f121f5bddff6caa683d09422232b04159ed0...,,2023-09-26
4068,,,1961,organisation,default-value,1962,listed-building-outline,bf55d2bbd7f121f5bddff6caa683d09422232b04159ed0...,,local-authority-eng:CAM


#### Final dataset comparison against the sqlite database

Below are two tables which show the difference betwen what was provided to us and what is ucrrently in the entity table. It is important to bear in mind that we assign entities automaticallyis process, the table above shows what we have added.

In [26]:
dataset_db = DatasetSqlite(os.path.join(data_dir,'dataset',f'{dataset}.sqlite3'))
results = dataset_db.get_entities()
results

,dataset,end_date,entity,entry_date,geojson,geometry,json,name,organisation_entity,point,prefix,reference,start_date,typology
0,listed-building-outline,,42115853,2023-09-26,,"MULTIPOLYGON (((-0.129989 51.535897,-0.130021 ...","{""listed-building"": ""10271"", ""listed-building-...","(East, off) Court Building, St Pancras Coroner...",71,POINT(-0.130051 51.535969),listed-building-outline,LB1859,2003-09-05,geography
1,listed-building-outline,,42115854,2023-09-26,,"MULTIPOLYGON (((-0.192568 51.561490,-0.192569 ...","{""listed-building"": ""477772"", ""listed-building...",(West side) Cattle Trough at junction with Her...,71,POINT(-0.192542 51.561486),listed-building-outline,LB1481,1998-07-01,geography
2,listed-building-outline,,42115855,2023-09-26,,POINT (-0.164509 51.572476),"{""listed-building"": ""478043"", ""listed-building...",(South side) K6 Telephone kiosk,71,POINT(-0.164509 51.572476),listed-building-outline,LB173,1987-07-31,geography
3,listed-building-outline,,42115856,2023-09-26,,"MULTIPOLYGON (((-0.174260 51.541968,-0.174272 ...","{""listed-building"": ""492770"", ""listed-building...","HAMPSTEAD, ADELAIDE ROAD Swiss Cottage Regency...",71,POINT(-0.174453 51.542105),listed-building-outline,LB1872,2006-09-18,geography
4,listed-building-outline,,42115857,2023-09-26,,"MULTIPOLYGON (((-0.134375 51.520073,-0.134312 ...","{""listed-building"": ""1061382"", ""listed-buildin...","Nos. 64-67, Nos. 2-8",71,POINT(-0.134257 51.520167),listed-building-outline,LB1531,2002-07-19,geography
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1956,listed-building-outline,,42117809,2023-09-26,,"MULTIPOLYGON (((-0.190240 51.546114,-0.190323 ...","{""listed-building"": ""1475683"", ""listed-buildin...",Lilian Baylis House (former Decca recording st...,71,POINT(-0.190384 51.546078),listed-building-outline,LB1983,2021-08-06,geography
1957,listed-building-outline,,42117810,2023-09-26,,"MULTIPOLYGON (((-0.125078 51.516971,-0.125123 ...","{""listed-building"": ""1485008"", ""listed-buildin...","35 and 37 New Oxford Street, London, WC1A 1BH",71,POINT(-0.125167 51.517000),listed-building-outline,LB1985,2023-02-23,geography
1958,listed-building-outline,,42117811,2023-09-26,,"MULTIPOLYGON (((-0.125065 51.516917,-0.125054 ...","{""listed-building"": ""1485009"", ""listed-buildin...","10-12 Museum Street, London, WC1A 1JS",71,POINT(-0.124994 51.516892),listed-building-outline,LB1986,2023-02-23,geography
1959,listed-building-outline,,42117812,2023-09-26,,"MULTIPOLYGON (((-0.125084 51.516946,-0.124977 ...","{""listed-building"": ""1486033"", ""listed-buildin...","The Old Crown, 33 New Oxford Street, London, W...",71,POINT(-0.125051 51.517004),listed-building-outline,LB1987,2023-06-20,geography


In [27]:
# load in raw resources
collection = Collection(os.path.join(data_dir,'collection'))
collection.load(directory=os.path.join(data_dir,'collection'))
resources = collection.resource.entries
resources

[{'resource': 'bf55d2bbd7f121f5bddff6caa683d09422232b04159ed01db4e8f3922e2d2660',
  'bytes': '',
  'organisations': 'local-authority-eng:CAM',
  'datasets': 'listed-building-outline',
  'endpoints': '431efe2f2cbc73eb181f67ce9cc27cc61f88236835eb2b3b96ee6a54ceb6458a',
  'start-date': '2023-09-26',
  'end-date': ''}]

In [28]:
# currently this just reads in the raw resource but in  the future this should check for 
# converted resource first
resource = resources[0]['resource']
resource_path = os.path.join(data_dir,'collection','resource',resource)
try:
    raw_resource = pd.read_csv(resource_path)
except (UnicodeDecodeError,TypeError,pd.errors.ParserError):
    converted_resource_dir = os.path.join(data_dir,'var','converted_resources')
    converted_resource_path = os.path.join(converted_resource_dir,f'{resource}.csv') 
    if not os.path.exists(converted_resource_path):
        convert_resource(resource,resource_path,converted_resource_dir,dataset)
    raw_resource = pd.read_csv(converted_resource_path)
    

In [29]:
raw_resource

,reference,name,listed-building,listed-building-grade,notes,start-date,end-date,entry-date,geometry
0,LB1859,"(East, off) Court Building, St Pancras Coroner...",10271,II,NaN,05/09/2003,NaN,NaN,"POLYGON ((-0.129989 51.535897,-0.130021 51.535..."
1,LB1481,(West side) Cattle Trough at junction with Her...,477772,II,NaN,01/07/1998,NaN,NaN,"POLYGON ((-0.192568 51.56149,-0.192569 51.5614..."
2,LB173,(South side) K6 Telephone kiosk,478043,II,NaN,31/07/1987,NaN,NaN,POINT (-0.164509 51.572476)
3,LB1872,"HAMPSTEAD, ADELAIDE ROAD Swiss Cottage Regency...",492770,II,NaN,18/09/2006,NaN,NaN,"POLYGON ((-0.17426 51.541968,-0.174269 51.5419..."
4,LB1531,"Nos. 64-67, Nos. 2-8",1061382,II,NaN,19/07/2002,NaN,NaN,"POLYGON ((-0.134375 51.520073,-0.134506 51.520..."
...,...,...,...,...,...,...,...,...,...
1956,LB1983,Lilian Baylis House (former Decca recording st...,1475683,II,NaN,06/08/2021,NaN,NaN,"POLYGON ((-0.19024 51.546114,-0.190323 51.5461..."
1957,LB1985,"35 and 37 New Oxford Street, London, WC1A 1BH",1485008,II,NaN,23/02/2023,NaN,NaN,"POLYGON ((-0.125078 51.516971,-0.125123 51.517..."
1958,LB1986,"10-12 Museum Street, London, WC1A 1JS",1485009,II,NaN,23/02/2023,NaN,NaN,"POLYGON ((-0.125065 51.516917,-0.125084 51.516..."
1959,LB1987,"The Old Crown, 33 New Oxford Street, London, W...",1486033,II,NaN,20/06/2023,NaN,NaN,"POLYGON ((-0.125084 51.516946,-0.124979 51.516..."


In [30]:
a_list= ['a',2]

In [31]:
a_list

['a', 2]